In [1]:
# test to see if we can load these in buildingmotif 
from buildingmotif import BuildingMOTIF

from buildingmotif.dataclasses import Library, Model, Template
from rdflib import Graph

from semantic_objects.s223 import *
from semantic_objects.core import export_templates

bm = BuildingMOTIF("sqlite://")

CRITICAL:root:Install the 'bacnet-ingress' module, e.g. 'pip install buildingmotif[bacnet-ingress]'
DEBUG:buildingmotif.database.graph_connection:Creating tables for graph storage


In [2]:
get_related_classes(Space)

([semantic_objects.s223.relations.hasProperty,
  semantic_objects.s223.relations.hasUnit,
  semantic_objects.s223.relations.hasValue,
  semantic_objects.s223.relations.hasQuantityKind],
 [semantic_objects.s223.properties.Area,
  semantic_objects.s223.entities.Space,
  semantic_objects.qudt.quantitykinds.QuantityKind,
  semantic_objects.units.Unit])

In [3]:

lib = Library.create('test')

DEBUG:buildingmotif.database.table_connection:Creating shape collection in library: 'test'
DEBUG:buildingmotif.database.table_connection:Creating database library: 'test'


In [4]:
templates = {}
for lst in get_related_classes(Space):
    for obj in lst:
        try:
            templates[obj.__name__] = lib.create_template(obj.__name__)
            templates[obj.__name__].body.parse(data=obj.generate_turtle_body(), format='turtle')
        except Exception as e:
            print('already has template', obj.__name__)

for lst in get_related_classes(Space):
    for obj in lst:
        for dependency in obj.get_dependencies():
            print('adding dependency for class ', obj.__name__, dependency['template'])
            templates[obj.__name__].add_dependency(templates[dependency['template'].__name__], dependency['args'])


DEBUG:buildingmotif.database.table_connection:Creating database template: 'hasProperty'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '0162c316-9689-4639-85d1-1fe8258d493b' in database with: 0 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'hasUnit'
DEBUG:buildingmotif.database.graph_connection:Creating graph: 'd135ab5d-614d-48c9-948c-bdeb771a52be' in database with: 0 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'hasValue'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '3cb39a85-c05f-4fe3-a84c-9acf72943ce0' in database with: 0 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'hasQuantityKind'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '9093ab90-0b34-4153-921f-91db5ef98aed' in database with: 0 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'Area'
DEBUG:buildingmotif.database.graph_connection:Creating

adding dependency for class  Space <class 'semantic_objects.s223.properties.Area'>


In [5]:
templates['Area']

Template(_id=5, _name='Area', body=<Graph identifier=36ed7ad9-bbc4-4d92-be57-fc96fe663187 (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x10416cd10>)

In [6]:
bm.session.commit()

In [7]:
lib.get_template_by_name('Space').check_dependencies()

DEBUG:buildingmotif.database.table_connection:Creating depencency from templates with ids: '6' and: '5'


In [8]:
help(lib.get_template_by_name('Space').add_dependency)

Help on method add_dependency in module buildingmotif.dataclasses.template:

add_dependency(dependency: 'Template', args: Dict[str, str]) -> None method of buildingmotif.dataclasses.template.Template instance
    Add dependency to template.
    
    :param dependency: dependency to add
    :type dependency: Template
    :param args: dictionary of dependency arguments
    :type args: Dict[str, str]



In [9]:
templates['Area'].body.print()

@prefix P: <urn:___param___#> .
@prefix quantitykind: <http://qudt.org/vocab/quantitykind/> .
@prefix s223: <http://data.ashrae.org/standard223#> .

P:name a s223:Area ;
    s223:hasQuantityKind quantitykind:Area ;
    s223:hasUnit P:unit ;
    s223:hasValue P:value .




In [10]:
templates['Area'].parameters

{'name', 'unit', 'value'}

In [11]:
lib.get_template_by_name('Space').inline_dependencies().body.print()

@prefix ns1: <http://data.ashrae.org/standard223#> .

<urn:___param___#name> a ns1:Space ;
    ns1:hasProperty <urn:___param___#area> .

<urn:___param___#area> a ns1:Area ;
    ns1:hasQuantityKind <http://qudt.org/vocab/quantitykind/Area> ;
    ns1:hasUnit <urn:___param___#area-unit> ;
    ns1:hasValue <urn:___param___#area-value> .




In [12]:
bm.session.commit()

In [13]:
lib.get_template_by_name('Space').add_dependency(lib.get_template_by_name('Area'), {'name': 'area'})

DEBUG:buildingmotif.database.table_connection:Creating depencency from templates with ids: '6' and: '5'


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: deps_association_table.dependant_id, deps_association_table.dependee_id, deps_association_table.args
[SQL: INSERT INTO deps_association_table (dependant_id, dependee_id, args) VALUES (?, ?, ?)]
[parameters: (6, 5, '[["name", "area"]]')]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

In [ ]:
templates['Space'].body.print()

@prefix P: <urn:___param___#> .
@prefix s223: <http://data.ashrae.org/standard223#> .

P:name a s223:Space ;
    s223:hasProperty P:area .




In [ ]:
templates['Area'].body.print()

@prefix P: <urn:___param___#> .
@prefix quantitykind: <http://qudt.org/vocab/quantitykind/> .
@prefix s223: <http://data.ashrae.org/standard223#> .

P:name a s223:Area ;
    s223:hasQuantityKind quantitykind:Area ;
    s223:hasUnit P:unit ;
    s223:hasValue P:value .




In [ ]:
templates['Area'].check_dependencies()

In [ ]:
lib.get_template_by_name('Space').transitive_parameters

{'area', 'area-unit', 'area-value', 'name'}

In [ ]:
help(lib.get_template_by_name('Space').check_dependencies)

Help on method check_dependencies in module buildingmotif.dataclasses.template:

check_dependencies() method of buildingmotif.dataclasses.template.Template instance
    Verifies that all dependencies have valid references to the parameters
    in the dependency or (recursively) its dependencies. Raises an exception if any
    issues are found.
    
    It is recommended to call this after *all* templates in a library have been
    loaded in to the DB, else this might falsely raise an error.



In [ ]:
help(lib)

Help on Library in module buildingmotif.dataclasses.library object:

class Library(builtins.object)
 |  Library(_id: int, _name: str, _bm: 'BuildingMOTIF') -> None
 |  
 |  This class mirrors :py:class:`database.tables.DBLibrary`.
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __init__(self, _id: int, _name: str, _bm: 'BuildingMOTIF') -> None
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  create_template(self, name: str, body: Optional[rdflib.graph.Graph] = None, optional_args: Optional[List[str]] = None) -> buildingmotif.dataclasses.template.Template
 |      Create template in this library.
 |      
 |      :param name: name
 |      :type name: str
 |      :param body: template body
 |      :type body: rdflib.Graph
 |      :param optional_args: optional parameters for the template
 |      :type optional_args: list[str]
 |      :return: created template


In [ ]:
templates['Area'].body.print()

@prefix P: <urn:___param___#> .
@prefix quantitykind: <http://qudt.org/vocab/quantitykind/> .
@prefix s223: <http://data.ashrae.org/standard223#> .

P:name a s223:Area ;
    s223:hasQuantityKind quantitykind:Area ;
    s223:hasUnit P:unit ;
    s223:hasValue P:value .




In [ ]:
templates['Space'].inline_dependencies().body.print()

@prefix ns1: <http://data.ashrae.org/standard223#> .

<urn:___param___#name> a ns1:Space ;
    ns1:hasProperty <urn:___param___#area> .

<urn:___param___#area> a ns1:Area ;
    ns1:hasQuantityKind <http://qudt.org/vocab/quantitykind/Area> ;
    ns1:hasUnit <urn:___param___#area-unit> ;
    ns1:hasValue <urn:___param___#area-value> .




In [ ]:
dependency['template'].__name__

NameError: name 'dependency' is not defined

In [ ]:
space_template = lib.create_template('space')

DEBUG:buildingmotif.database.table_connection:Creating database template: 'space'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '401e3a68-74cc-4063-820e-c62b78d1f5f6' in database with: 0 triples


In [ ]:
space_template.body = Graph().parse(data = Space.generate_turtle_body(), format='turtle')


In [ ]:
get_related

NameError: name 'get_related' is not defined

In [ ]:
for dep in Space.get_dependencies():
    dep_template = lib.create_template(dep.__name__)
    dep_template.body = Graph().parse(data=dep.generate_turtle_body(), format='turtle')

for dep in Space.get_dependencies():
    

AttributeError: 'Template' object has no attribute 'dependenc'

In [ ]:
ex_template_dir = "/Users/lazlopaul/Desktop/HPflex/Semantic_MPC_Interface/src/semantic_mpc_interface/templates/brick-templates"

In [ ]:
ex_templates = Library.load(directory=ex_template_dir)

In [ ]:
templates.get_templates()

[]

In [ ]:
ex_templates.get_templates()

[Template(_id=1, _name='hvac-zone', body=<Graph identifier=5b3d6c2f-172d-49cb-b8cc-9fe14b0cdbf0 (<class 'rdflib.graph.Graph'>)>, optional_args=['space', 'window', 'tstat', 'hvac'], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x11a07f910>),
 Template(_id=2, _name='space', body=<Graph identifier=0a5c2d9d-6c2a-4a89-8973-7ffa82dbf2f5 (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x11a07f910>),
 Template(_id=3, _name='window', body=<Graph identifier=7eb8fcef-af22-4ed2-b667-7ed84816b16a (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x11a07f910>),
 Template(_id=4, _name='hp-rtu', body=<Graph identifier=b3444f99-dc44-4b50-b5da-59739754b078 (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x11a07f910>),
 Template(_id=5, _name='tstat', body

In [ ]:
ex_template_2 = Library.load(directory = 'ex-templates')

In [ ]:
ex_template_2.get_templates()

[Template(_id=45, _name='area', body=<Graph identifier=1a0aa440-a25d-41a3-9834-5c8576d5e0cf (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x11a07f910>),
 Template(_id=46, _name='tilt', body=<Graph identifier=1f304be5-345c-4668-9c8b-20a628e76731 (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x11a07f910>),
 Template(_id=47, _name='azimuth', body=<Graph identifier=017ac19b-f132-45f4-8213-9a2c28a67ec0 (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x11a07f910>),
 Template(_id=48, _name='cooling-COP', body=<Graph identifier=dd4bf7f4-bfaf-441d-b3f4-03c3d4b5c9d2 (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x11a07f910>),
 Template(_id=49, _name='heating-COP', body=<Graph identifier=0091